In [3]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
import ssl
import os

os.makedirs('preview', exist_ok=True) 

ssl._create_default_https_context = ssl._create_unverified_context


#CIFAR_10 set of 60k images 32x32 pixels on 3 channels
IMG_CHANNELS = 3

IMG_ROWS = 32
IMG_COLS = 32

#CONSTANTS
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# # Remove after testing
# X_train = X_train[:10000]
# y_train = y_train[:10000]
# X_test = X_test[:2000]
# y_test = y_test[:2000]
# #######################

print("X_train shape: ", X_train.shape)
print(X_train.shape[0], " train samples")
print(X_test.shape[0], " test samples")

#convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

#float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', 
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))

model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)

from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
NUM_TO_AUGMENT = 5

#load dataset
#(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# augmenting
print("Augmenting training set images...")

print("X_train shape: ", X_train.shape)
print(X_train.shape[0], " train samples")
print(X_test.shape[0], " test samples")

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
    for x_aug in datagen.flow(x, batch_size=1, 
                              save_to_dir='preview', 
                              save_prefix='cifar', 
                              save_format='jpeg'):
        if num_aug >= NUM_TO_AUGMENT:
            break
        xtas.append(x_aug[0])
        ytas.append(y_train[i]) # assuming that y_train is available
        num_aug += 1

print("Before Fitting the datagen")
print("X_train shape: ", X_train.shape)
print(X_train.shape[0], " train samples")
print(X_test.shape[0], " test samples")

#fit the datagen
datagen.fit(X_train)

print("Before Train")
print("X_train shape: ", X_train.shape)
print(X_train.shape[0], " train samples")
print(X_test.shape[0], " test samples")

#train
history = model.fit_generator(
    datagen.flow(
        X_train, 
        Y_train, 
        batch_size=BATCH_SIZE),
    samples_per_epoch=X_train.shape[0],
    epochs=NB_EPOCH,
    verbose=VERBOSE)

score = model.evaluate(
    X_test,
    Y_test, 
    batch_size=BATCH_SIZE, 
    verbose=VERBOSE)
print("Test score: ", score[0])
print("Test accuracy: ", score[1])

X_train shape:  (50000, 32, 32, 3)
50000  train samples
10000  test samples
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 16, 32)        0         
____________________________________________

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:164: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=20, verbose=1, steps_per_epoch=390)`


Epoch 1/20
390/390 [==============================] - 117s 300ms/step - loss: 1.8664 - accuracy: 0.3137
Epoch 2/20
390/390 [==============================] - 120s 307ms/step - loss: 1.6055 - accuracy: 0.4179
Epoch 3/20
390/390 [==============================] - 118s 303ms/step - loss: 1.4846 - accuracy: 0.4620
Epoch 4/20
390/390 [==============================] - 117s 299ms/step - loss: 1.4107 - accuracy: 0.4899
Epoch 5/20
390/390 [==============================] - 122s 313ms/step - loss: 1.3474 - accuracy: 0.5154
Epoch 6/20
390/390 [==============================] - 117s 301ms/step - loss: 1.3097 - accuracy: 0.5320
Epoch 7/20
390/390 [==============================] - 120s 307ms/step - loss: 1.2700 - accuracy: 0.5428
Epoch 8/20
390/390 [==============================] - 119s 305ms/step - loss: 1.2487 - accuracy: 0.5536
Epoch 9/20
390/390 [==============================] - 118s 303ms/step - loss: 1.2143 - accuracy: 0.5644
Epoch 10/20
390/390 [==============================] - 130s 334m

# How this algorithm could result in ethical and privacy concerns if it were trained on different sets of images.

Different sets of images could have several ethical or privacy concerns. Some that come to mind are:
- If trained on images of humans, there may be some kind of bias toward a certain demographic or group of people. The model may learn to recognize patterns that are specific to that group, leading to unfair outcomes when applied to other groups.
- It may learn to recognize features that are not relevant to the intended classification task, but are still present in the training data. For example, if the training dataset contains images of only GREEN frogs, then it may determine that a blue and orange frog is not a frog at all and may try to place it into some other classification.
- 